In [1]:
import pandas as pd
fixedData = pd.read_csv('Cleaned-Data.csv')

In [2]:
relevant_data = fixedData[['text','Label']]

In [3]:
import numpy as np
train_indices = np.random.rand(len(relevant_data)) < 0.1
train = relevant_data[train_indices].reset_index(drop=True)
test = relevant_data[~train_indices].reset_index(drop=True)
test_indices = np.random.rand(len(test)) < 0.7
val = test[~test_indices].reset_index(drop=True)
test = test[test_indices].reset_index(drop=True)
print(train.head())
print(len(train))
print(test.head())
print(len(test))
print(val.head())
print(len(val))

                                                text  Label
0  ['trump', 'on', 'twitter', 'dec', 'global', 'w...      0
1  ['trump', 'travel', 'ban', 'should', 'not', 'a...      0
2  ['trump', 'on', 'twitter', 'dec', 'tax', 'cut'...      0
3  ['as', 'republicans', 'aim', 'to', 'ride', 'ec...      0
4  ['senate', 'shelves', 'disaster', 'aid', 'bill...      0
4424
                                                text  Label
0  ['as', 'us', 'budget', 'fight', 'looms', 'repu...      0
1  ['senior', 'us', 'republican', 'senator', 'let...      0
2  ['fbi', 'russia', 'probe', 'helped', 'by', 'au...      0
3  ['trump', 'wants', 'postal', 'service', 'to', ...      0
4  ['white', 'house', 'congress', 'prepare', 'for...      0
28396
                                                text  Label
0  ['us', 'military', 'to', 'accept', 'transgende...      0
1  ['man', 'says', 'he', 'delivered', 'manure', '...      0
2  ['us', 'lawmakers', 'question', 'businessman',...      0
3  ['exclusive', 'us', 'memo'

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
v = CountVectorizer()
training_text = []
training_result = []
for i in range(len(train)): #len(train)
    training_text.append(''.join(str(x) for x in train['text'][i]))
    training_result.append(train['Label'][i])
training_text = v.fit_transform(training_text)

In [5]:
text = []
result = []
for i in range(len(train)):
    text.append(''.join(str(x) for x in train['text'][i]))
    result.append(train['Label'][i])
X_train = v.transform(text)
y_train = np.asarray(result)

text = []
result = []
for i in range(len(val)):
    text.append(''.join(str(x) for x in val['text'][i]))
    result.append(val['Label'][i])
X_val = v.transform(text)
y_val = np.asarray(result)

text = []
result = []
for i in range(len(test)):
    text.append(''.join(str(x) for x in test['text'][i]))
    result.append(test['Label'][i])
X_test = v.transform(text)
y_test = np.asarray(result)

In [6]:
from tensorflow.python.keras.preprocessing import sequence
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout, Activation
from tensorflow.python.keras.layers import Embedding
from tensorflow.python.keras.layers import Conv1D, GlobalMaxPooling1D
# set parameters:
max_features = 5000
maxlen = 400
batch_size = 32
embedding_dims = 50
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 2

model = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model.add(Embedding(max_features,
                    embedding_dims,
                    input_length=X_train.shape[1]))
model.add(Dropout(0.2))

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
# we use max pooling:
model.add(GlobalMaxPooling1D())

# We add a vanilla hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 60935, 50)         250000    
_________________________________________________________________
dropout (Dropout)            (None, 60935, 50)         0         
_________________________________________________________________
conv1d (Conv1D)              (None, 60933, 250)        37750     
_________________________________________________________________
global_max_pooling1d (Global (None, 250)               0         
_________________________________________________________________
dense (Dense)                (None, 250)               62750     
_________________________________________________________________
dropout_1 (Dropout)          (None, 250)               0         
_________________________________________________________________
activation (Activation)      (None, 250)               0         
__________

In [ ]:
history = model.fit(X_train, y_train, epochs=3, validation_data=(X_val, y_val), batch_size=10)
loss, accuracy = model.evaluate(X_train, y_train)
print('Training Accuracy: {:.4f}'.format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test)
print("Testing Accuracy: {:.4f}".format(accuracy))

/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 4424 samples, validate on 12078 samples
Epoch 1/3
2680/4424 [=================>............] - ETA: 53:42 - loss: 0.6923 - acc: 0.5183